In [1]:
import pandas as pd
import numpy as np
import scipy.stats
import re
import requests
import math
from bs4 import BeautifulSoup
import matplotlib as mlp
import matplotlib.pyplot as plt
import matplotlib.backends.backend_agg
import matplotlib.figure
import seaborn as sb
import datetime
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, LabelBinarizer
from sklearn.metrics import roc_curve, roc_auc_score

In [2]:
#PRICES SCRAPING

In [ ]:
fst_page_listings_soup = BeautifulSoup(requests.get('https://www.truecar.com/used-cars-for-sale/listings/').content, 'lxml')

In [3]:
page1_prices_soup = fst_page_listings_soup.find_all('h4', {'data-test': 'vehicleCardPricingBlockPrice'})
page1_prices_soup

[<h4 class="heading-3 margin-y-1 font-weight-bold" data-qa="Heading" data-test="vehicleCardPricingBlockPrice">$16,277</h4>,
 <h4 class="heading-3 margin-y-1 font-weight-bold" data-qa="Heading" data-test="vehicleCardPricingBlockPrice">$7,099</h4>,
 <h4 class="heading-3 margin-y-1 font-weight-bold" data-qa="Heading" data-test="vehicleCardPricingBlockPrice">$14,999</h4>,
 <h4 class="heading-3 margin-y-1 font-weight-bold" data-qa="Heading" data-test="vehicleCardPricingBlockPrice">$26,500</h4>,
 <h4 class="heading-3 margin-y-1 font-weight-bold" data-qa="Heading" data-test="vehicleCardPricingBlockPrice">$7,995</h4>,
 <h4 class="heading-3 margin-y-1 font-weight-bold" data-qa="Heading" data-test="vehicleCardPricingBlockPrice">$26,419</h4>,
 <h4 class="heading-3 margin-y-1 font-weight-bold" data-qa="Heading" data-test="vehicleCardPricingBlockPrice">$8,000</h4>,
 <h4 class="heading-3 margin-y-1 font-weight-bold" data-qa="Heading" data-test="vehicleCardPricingBlockPrice">$28,300</h4>,
 <h4 class=

In [4]:
#ANOTHER WAY TO SCRAPE THE PRICES
#listings_soup = BeautifulSoup(requests.get('https://www.truecar.com/used-cars-for-sale/listings/?page=2').content, 'lxml')
#listings_soup.find_all('h4', {'class': 'heading-3 margin-y-1 font-weight-bold'})

In [5]:
#YEARS SCRAPING

In [6]:
fst_page_listings_years = fst_page_listings_soup.find_all('span', {'class': 'vehicle-card-year'})
fst_page_years = list(re.findall('[12][0-9]{3}', str(fst_page_listings_years)))
print(len(fst_page_years))
fst_page_years

33


['2014',
 '2018',
 '2019',
 '2018',
 '2016',
 '2016',
 '2017',
 '2018',
 '2017',
 '2018',
 '2017',
 '2019',
 '2018',
 '2018',
 '2018',
 '2017',
 '2017',
 '2018',
 '2016',
 '2016',
 '2016',
 '2016',
 '2017',
 '2017',
 '2016',
 '2017',
 '2019',
 '2017',
 '2016',
 '2017',
 '2018',
 '2017',
 '2017']

In [7]:
def pageyears(number):
    nth_page_listings_soup = BeautifulSoup(requests.get('https://www.truecar.com/used-cars-for-sale/listings/?page=' + str(number)).content, 'lxml')
    nth_page_listings = nth_page_listings_soup.find_all('span', {'class': 'vehicle-card-year'})
    nth_page_years = list(re.findall('[12][0-9]{3}', str(nth_page_listings)))
    return nth_page_years

In [8]:
rest_page_years1 = []
for number_of_page in range(2, 180):
    rest_page_years1 += pageyears(number_of_page)

In [9]:
rest_page_years2 = []
for number_of_page in range(180, 360):
    rest_page_years2 += pageyears(number_of_page)

In [ ]:
rest_page_years3 = []
for number_of_page in range(360, 540):
    rest_page_years3 += pageyears(number_of_page)

In [10]:
years = fst_page_years + rest_page_years1 + rest_page_years2 + rest_page_years3
print(len(years))
years

16173


['2014',
 '2018',
 '2019',
 '2018',
 '2016',
 '2016',
 '2017',
 '2018',
 '2017',
 '2018',
 '2017',
 '2019',
 '2018',
 '2018',
 '2018',
 '2017',
 '2017',
 '2018',
 '2016',
 '2016',
 '2016',
 '2016',
 '2017',
 '2017',
 '2016',
 '2017',
 '2019',
 '2017',
 '2016',
 '2017',
 '2018',
 '2017',
 '2017',
 '2017',
 '2018',
 '2018',
 '2017',
 '2018',
 '2009',
 '2017',
 '2017',
 '2017',
 '2015',
 '2019',
 '2018',
 '2017',
 '2014',
 '2017',
 '2016',
 '2016',
 '2015',
 '2017',
 '2017',
 '2018',
 '2017',
 '2019',
 '2016',
 '2018',
 '2017',
 '2017',
 '2017',
 '2019',
 '2017',
 '2017',
 '2016',
 '2016',
 '2017',
 '2018',
 '2015',
 '2015',
 '2017',
 '2017',
 '2018',
 '2012',
 '2012',
 '2017',
 '2017',
 '2017',
 '2018',
 '2005',
 '2017',
 '2015',
 '2012',
 '1999',
 '2017',
 '2017',
 '2015',
 '2016',
 '2019',
 '2012',
 '2017',
 '2017',
 '1999',
 '2017',
 '2017',
 '2013',
 '2007',
 '2017',
 '2008',
 '2019',
 '2018',
 '2015',
 '2017',
 '2002',
 '2017',
 '2017',
 '2017',
 '2017',
 '2017',
 '2016',
 '2016',
 

In [11]:
#MILEAGE SCRAPING

In [12]:
fst_page_listings_soup = BeautifulSoup(requests.get('https://www.truecar.com/used-cars-for-sale/listings/').content, 'lxml')
fst_page_listings_miles = page1_listings_soup.find_all('div', {'class': 'font-size-1 text-truncate'})
fst_page_listings_miles

[<div class="font-size-1 text-truncate" data-test="vehicleCardTrim">LX Sedan CVT</div>,
 <div class="font-size-1 text-truncate">At or near <!-- -->avg. list price</div>,
 <div class="font-size-1 text-truncate" data-test="vehicleMileage"><svg class="icon icon-before vehicle-card-icon icon-color-default" data-qa="IconSpeedometer" style="width:16px;height:16px;stroke-width:1.5" viewbox="0 0 24 24"><path d="M.28 15.67l2.68.19M1.04 12.26l2.59 1M2.77 9.22l2.24 1.76M5.32 6.82l1.57 2.26M8.47 5.28l3.21 10.58M11.92 4.74l.02 2.9M15.38 5.23l-.83 2.79M18.55 6.73l-1.62 2.41M21.13 9.09l-2.24 1.81M22.91 12.1l-2.64 1.06M23.72 15.5l-2.8.24"></path><circle cx="11.92" cy="16.64" r=".87"></circle></svg>32,363<!-- --> miles</div>,
 <div class="font-size-1 text-truncate" data-test="vehicleCardTrim">SE Sedan Auto</div>,
 <div class="font-size-1 text-truncate">$<!-- -->873<!-- --> <!-- -->off<!-- --> <!-- -->avg. list price</div>,
 <div class="font-size-1 text-truncate" data-test="vehicleMileage"><svg class="i

In [13]:
fst_page_miles = re.findall('g>[0-9]*,[0-9]+', str(fst_page_listings_miles))
fst_page_miles

['g>32,363',
 'g>52,509',
 'g>29,607',
 'g>43,503',
 'g>60,227',
 'g>47,692',
 'g>185,354',
 'g>15,681',
 'g>152,738',
 'g>42,508',
 'g>99,515',
 'g>21,019',
 'g>79,030',
 'g>41,244',
 'g>148,518',
 'g>34,896',
 'g>64,490',
 'g>30,924',
 'g>100,492',
 'g>31,445',
 'g>50,535',
 'g>37,409',
 'g>80,026',
 'g>53,620',
 'g>103,293',
 'g>56,900',
 'g>14,131',
 'g>46,020',
 'g>77,223',
 'g>72,300',
 'g>33,645',
 'g>143,192',
 'g>46,922']

In [14]:
fst_page_mileage = list(map(lambda mileage: mileage[2:], fst_page_miles))
fst_page_mileage

['32,363',
 '52,509',
 '29,607',
 '43,503',
 '60,227',
 '47,692',
 '185,354',
 '15,681',
 '152,738',
 '42,508',
 '99,515',
 '21,019',
 '79,030',
 '41,244',
 '148,518',
 '34,896',
 '64,490',
 '30,924',
 '100,492',
 '31,445',
 '50,535',
 '37,409',
 '80,026',
 '53,620',
 '103,293',
 '56,900',
 '14,131',
 '46,020',
 '77,223',
 '72,300',
 '33,645',
 '143,192',
 '46,922']

In [15]:
def pagemileage(number):
    nth_listings_soup_miles = BeautifulSoup(requests.get('https://www.truecar.com/used-cars-for-sale/listings/?page=' + str(number)).content, 'lxml')
    nth_page_listings_miles = page1_listings_soup.find_all('div', {'class': 'font-size-1 text-truncate'})
    nth_page_miles = re.findall('g>[0-9]*,[0-9]+', str(fst_page_listings_miles))
    return nth_page_miles

In [16]:
rest_page_mileage1 = []
for number_of_page in range(2, 180):
    rest_page_mileage1 += pagemileage(number_of_page)

In [17]:
rest_page_mileage2 = []
for number_of_page in range(180, 360):
    rest_page_mileage2 += pagemileage(number_of_page)

In [ ]:
rest_page_mileage3 = []
for number_of_page in range(360, 540):
    rest_page_mileage3 += pagemileage(number_of_page)

In [ ]:
page_mileage = fst_page_mileage + rest_page_mileage1 + rest_page_mileage2 + rest_page_mileage3
mileage = list(map(lambda miles: miles[2:], page_mileage))
print(len(mileage))
mileage